In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import wandb
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import PredefinedSplit

In [2]:
import sys
sys.path.append('../../../../code')

from metrics import get_hi_metrics

Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/home/steshin/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [3]:
train = pd.read_csv('../../../../data/hi/hiv/train_1.csv', index_col=0)
test = pd.read_csv('../../../../data/hi/hiv/test_1.csv', index_col=0)

train

,smiles,value
4,O=S(=O)(O)CCS(=O)(=O)O,0
21,CC(C)CCS(=O)(=O)O,0
90,O=S(=O)(O)CCO,0
106,O=S(=O)(O)CO,0
117,O=S(=O)(O)CCCCBr,0
...,...,...
40932,COC(=O)c1cc2cc3c(c(O)c2c(=O)o1)OC1(Oc2c(O)c4c(...,0
40973,CCCCC1C(OCOc2ccccc2)COC(=O)N1C(C)c1ccccc1,0
41024,CC(C)=CC1CC(C)C2CCC(C)C3C(=O)C(O)=C(C)C(=O)C123,0
41026,CCOC(=O)C12C(=O)C(C)CCC1C(C)CC2C=C(C)C,0


# Hyperparameter Optimization

In [4]:
def run_gb_gridsearch(train_fps, test_fps):
    split_index = [-1] * len(train_fps) + [0] * len(test_fps)
    pds = PredefinedSplit(test_fold = split_index)

    X = train_fps + test_fps
    y = train['value'].to_list() + test['value'].to_list()

    params = {
    'n_estimators': [10, 50, 100, 150, 200, 250, 500],
    'learning_rate': [0.01, 0.1, 0.3, 0.5, 0.7, 1.0],
    'subsample': [0.4, 0.7, 0.9, 1.0],
    'min_samples_split': [2, 3, 5, 7],
    'min_samples_leaf': [1, 3, 5],
    'max_depth': [2, 3, 4],
    'max_features': [None, 'sqrt']
    }
    knn = GradientBoostingClassifier()

    grid_search = RandomizedSearchCV(knn, params, cv=pds, n_iter=30, refit=False, scoring='average_precision', verbose=3)
    grid_search.fit(X, y)

    best_params = grid_search.best_params_
    print(best_params)
    knn = GradientBoostingClassifier(**best_params)
    knn.fit(train_fps, train['value'])

    test_preds = knn.predict_proba(test_fps)[:, 1]
    test_metrics = get_hi_metrics(test, test_preds)
    return test_metrics


In [5]:
train_mols = [Chem.MolFromSmiles(x) for x in train['smiles']]
train_morgan_fps = [AllChem.GetMorganFingerprintAsBitVect(x, 2, 1024) for x in train_mols]

test_mols = [Chem.MolFromSmiles(x) for x in test['smiles']]
test_morgan_fps = [AllChem.GetMorganFingerprintAsBitVect(x, 2, 1024) for x in test_mols]

test_metrics = run_gb_gridsearch(train_morgan_fps, test_morgan_fps)
print(test_metrics)

[09:42:02] WARNING: not removing hydrogen atom without neighbors
[09:42:02] WARNING: not removing hydrogen atom without neighbors


Fitting 1 folds for each of 30 candidates, totalling 30 fits
[CV 1/1] END learning_rate=0.1, max_depth=4, max_features=sqrt, min_samples_leaf=1, min_samples_split=3, n_estimators=100, subsample=0.4;, score=0.085 total time=  11.0s
[CV 1/1] END learning_rate=0.5, max_depth=2, max_features=sqrt, min_samples_leaf=3, min_samples_split=2, n_estimators=100, subsample=1.0;, score=0.108 total time=  10.8s
[CV 1/1] END learning_rate=0.5, max_depth=3, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=50, subsample=0.7;, score=0.114 total time=  10.2s
[CV 1/1] END learning_rate=1.0, max_depth=4, max_features=sqrt, min_samples_leaf=5, min_samples_split=3, n_estimators=250, subsample=0.4;, score=0.044 total time=  12.4s
[CV 1/1] END learning_rate=0.3, max_depth=3, max_features=None, min_samples_leaf=1, min_samples_split=7, n_estimators=150, subsample=0.4;, score=0.093 total time=  36.3s
[CV 1/1] END learning_rate=0.7, max_depth=3, max_features=None, min_samples_leaf=1, min_sa

# Final Evaluation

In [6]:
def fit_predict(train, test):
    train_mols = [Chem.MolFromSmiles(x) for x in train['smiles']]
    train_morgan_fps = [AllChem.GetMorganFingerprintAsBitVect(x, 2, 1024) for x in train_mols]

    test_mols = [Chem.MolFromSmiles(x) for x in test['smiles']]
    test_morgan_fps = [AllChem.GetMorganFingerprintAsBitVect(x, 2, 1024) for x in test_mols]

    gb = GradientBoostingClassifier(
        n_estimators=500,
        subsample=0.9,
        min_samples_split=3,
        min_samples_leaf=5,
        max_features=None,
        max_depth=3,
        learning_rate=0.1
    )
    gb.fit(train_morgan_fps, train['value'])

    train_result = train.copy()
    train_result['preds'] = train_result['value']

    test_result = test.copy()
    test_result['preds'] = gb.predict_proba(test_morgan_fps)[:, 1]

    return train_result, test_result


In [7]:
for i in [1, 2, 3]:
    train = pd.read_csv(f'../../../../data/hi/hiv/train_{i}.csv')
    test = pd.read_csv(f'../../../../data/hi/hiv/test_{i}.csv')

    train_preds, test_preds = fit_predict(train, test)
    train_preds.to_csv(f'../../../../predictions/hi/hiv/gb_ecfp4/train_{i}.csv')
    test_preds.to_csv(f'../../../../predictions/hi/hiv/gb_ecfp4/test_{i}.csv')


[11:04:38] WARNING: not removing hydrogen atom without neighbors
[11:04:38] WARNING: not removing hydrogen atom without neighbors
[11:08:09] WARNING: not removing hydrogen atom without neighbors
[11:08:09] WARNING: not removing hydrogen atom without neighbors
[11:11:31] WARNING: not removing hydrogen atom without neighbors
[11:11:31] WARNING: not removing hydrogen atom without neighbors
